In [75]:
import os
os.chdir('C:\\Users\\USUARIO\\Documents\\GitHub\\Team_224\\Proyecto\\Codigo\\Modelling')

In [89]:
u=!pip freeze
intalados=str(u)

In [77]:
to_instalar=['os','pandas','matplotlib','statsmodels','numpy','seaborn','pingouin','pycaret','sklearn','scipy','openpyxl']

In [80]:
instalados=[x for x in to_instalar if x in  intalados ]
if len(set(to_instalar)-set(instalados))>0:
    !pip install -r requirements_pycaret.txt 

  Using cached scipy-1.5.4-cp37-cp37m-win_amd64.whl (31.2 MB)
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pingouin-0.5.2-py3-none-any.whl
  Using cached outdated-0.2.1-py3-none-any.whl (7.5 kB)
  Using cached pandas_flavor-0.3.0-py3-none-any.whl (6.3 kB)
  Using cached pingouin-0.5.1-py3-none-any.whl
     -------------------------------------- 182.8/182.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 206.9/206.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 205.1/205.1 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pandas_flavor-0.2.0-p

In [84]:
import pycaret

In [85]:
#!pip install -r requirements.txt --user
import os
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from datetime import datetime
import math
import unicodedata
from unicodedata import normalize
import re
from dateutil.relativedelta import relativedelta
import pingouin
import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import scipy
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#import pycaret

In [86]:
ruta_madre="C:/Users/USUARIO/Documents/GitHub/Team_224/Proyecto"
ruta_insumos=os.path.join(ruta_madre, 'Insumos')
ruta_resultados=os.path.join(ruta_madre, 'Resultados')

## 1.0 Importando base de datos preprocesada

In [88]:
os.chdir(os.path.join(ruta_insumos,'preproccess_data'))
ruta_data_preproc=os.path.join(os.getcwd(),'data_agregada_preproccess.xlsx')
datos_preproc=pd.read_excel(ruta_data_preproc,index_col=0)


## 2.0 Featuring engineering 

Es empiza a hacer Feature engineering para evitar resultados raros

### 2.1 Deflactando los salarios

Debido al gran problema que genera la inflación a lo largo del tiempo, se prefiere usar cifras deflactadas, con lo cual, se utiliza la tabal abstraida del DANE para el IPC anual y se indexa hasta valores correspondientes al 2022. Es decir, la serie se deflacta y se pone todo en las mismas unidades monetarias en el mismo intervalo de tiempo

In [226]:
inflacion_variacion=pd.read_excel(os.path.join(os.getcwd(),'inflacion.xlsx'),index_col=0)
inflacion_variacion['inflacion']=inflacion_variacion['inflacion']/100+1
inflacion_variacion=inflacion_variacion[inflacion_variacion.index>=datos_preproc.year.min()]

In [229]:
inflacion_variacion=inflacion_variacion.sort_index(ascending=False).cumprod().reset_index()

In [231]:
datos_preproc=datos_preproc.merge(inflacion_variacion,on="year",how="left")
datos_preproc.inflacion.fillna(1,inplace=True)

In [233]:
datos_preproc['wage_deflacted']=datos_preproc['wage_imputed']*datos_preproc['inflacion']

### 2.2 Arreglando variables categoricas incompletas

In [237]:
datos_preproc['total_depedants']=datos_preproc['other_on_charge_person']+datos_preproc['children_amount']

In [238]:
datos_to_model=datos_preproc[['month','gender','is_special_population','any_disability','MUNICIPIO_DE_RESIDENCIA','home_type',
                             'education_level','PROFESION','marital_status','total_depedants','ESTRATO_SOCIAL','wage_deflacted',
                             'age','years_exp_current_role','request_attend_per_day']]
datos_to_model['total_depedants']=datos_to_model['total_depedants'].astype('str')
datos_to_model['ESTRATO_SOCIAL']=datos_to_model['ESTRATO_SOCIAL'].astype('str')

In [239]:
datos_to_model.isna().sum()

month                       0
gender                      3
is_special_population       3
any_disability              3
MUNICIPIO_DE_RESIDENCIA     3
home_type                   3
education_level             3
PROFESION                  65
marital_status              3
total_depedants             0
ESTRATO_SOCIAL              0
wage_deflacted              0
age                         3
years_exp_current_role      3
request_attend_per_day      0
dtype: int64

Note que hay un total importante de NAs en algunos datos. sin embargo, no tenemos ninguna explicación de negocio para reemplazarlos por un numero, por lo que se pondrá como una nueva categoria

In [240]:
datos_to_model['gender']=datos_to_model['gender'].fillna('OTHER')
datos_to_model['is_special_population']=datos_to_model['is_special_population'].fillna('NO APLICA')
datos_to_model['any_disability']=datos_to_model['any_disability'].fillna('no_reported')
datos_to_model['MUNICIPIO_DE_RESIDENCIA']=datos_to_model['MUNICIPIO_DE_RESIDENCIA'].fillna('NO_REPORTADO')
datos_to_model['home_type']=datos_to_model['home_type'].fillna('OTHER')
datos_to_model['education_level']=datos_to_model['education_level'].fillna('OTHER')
datos_to_model['PROFESION']=datos_to_model['PROFESION'].fillna('OTHER')
datos_to_model['marital_status']=datos_to_model['marital_status'].fillna('OTHER')
datos_to_model['total_depedants']=datos_to_model['total_depedants'].fillna('OTHER')
datos_to_model['ESTRATO_SOCIAL']=datos_to_model['ESTRATO_SOCIAL'].fillna('OTHER')

El siguiente codigo presenta una transformacion de la variable edad, ya que al verlo respecto a la variable que mide la productividad, tiene una relacion no lineal. 

### 2.3 Agregadno polinomios de la edad

In [241]:
datos_to_model['age2']=datos_to_model['age']**2
datos_to_model['age3']=datos_to_model['age']**3
datos_to_model['age4']=datos_to_model['age']**4
datos_to_model['age5']=datos_to_model['age']**5

### 2.1 Regresion lineal sin ninguna modificacion

#### 2.1.1 Dividiendo en test y train set

Utilzamos un split de 70/30, sin embargo, solo usaremos este ejercicio con fin de tener una referencia de potenciales variables significativas. siendo el peor modelo y luego ir viendo como se comporta a futuro

In [157]:
y=datos_to_model[datos_to_model.columns[datos_to_model.columns=='request_attend_per_day']]
X=datos_to_model[datos_to_model.columns[datos_to_model.columns!='request_attend_per_day']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#### 2.1.2 Normalizando y transformando las variables

Normalizaremos salario y años de experiencia y todas las variables de edad

In [158]:
scaler = StandardScaler()

stand_xtrain=scaler.fit_transform(X_train[['wage_imputed','years_exp_current_role','age','age2','age3','age4','age5']])
X_train[['wage_imputed','years_exp_current_role','age','age2','age3','age4','age5']]=pd.DataFrame(stand_xtrain,columns=['wage_imputed','years_exp_current_role','age','age2','age3','age4','age5'],
                                                                                                 index=X_train.index)

stand_xtest=scaler.fit_transform(X_test[['wage_imputed','years_exp_current_role','age','age2','age3','age4','age5']])
X_test[['wage_imputed','years_exp_current_role','age','age2','age3','age4','age5']]=pd.DataFrame(stand_xtest,columns=['wage_imputed','years_exp_current_role','age','age2','age3','age4','age5'],
                                                                                                index=X_test.index)


,wage_imputed,years_exp_current_role,age,age2,age3,age4,age5
0,-1.597724,1.126600,1.052010,0.896079,0.680317,0.453227,0.259047
1,-1.597724,-0.380445,0.111478,-0.023750,-0.125721,-0.182269,-0.199428
2,-1.597724,1.151158,1.052010,0.896079,0.680317,0.453227,0.259047
3,-1.597724,-0.355887,0.111478,-0.023750,-0.125721,-0.182269,-0.199428
4,-1.597724,1.176535,1.052010,0.896079,0.680317,0.453227,0.259047
...,...,...,...,...,...,...,...
1678,1.404478,-0.821671,-0.358788,-0.404821,-0.401644,-0.361535,-0.305721
1679,1.404478,0.253971,-0.123655,-0.220856,-0.273182,-0.281137,-0.259853
1680,1.953639,1.923099,0.346611,0.186497,0.042049,-0.062295,-0.121226
1681,1.953639,-0.268297,-1.064186,-0.877876,-0.686137,-0.515599,-0.382125


In [110]:
from pycaret.regression import *
s = setup(datos_to_model, target = 'request_attend_per_day')

,Description,Value
0,session_id,4146
1,Target,request_attend_per_day
2,Original Data,"(1683, 15)"
3,Missing Values,True
4,Numeric Features,3
5,Categorical Features,11
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1178, 61)"


In [109]:
best=compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,9.9619,277.5623,16.3857,0.7686,0.6124,2.9479,0.1190
lightgbm,Light Gradient Boosting Machine,11.2450,290.9522,16.8826,0.7556,0.7526,5.8501,0.0140
et,Extra Trees Regressor,11.1369,349.2361,18.5378,0.7036,0.6533,2.6865,0.1120
knn,K Neighbors Regressor,12.6769,419.0670,20.2712,0.6436,0.7477,8.4791,0.0200
dt,Decision Tree Regressor,11.6223,434.2818,20.3071,0.6391,0.7093,1.8738,0.0090
gbr,Gradient Boosting Regressor,15.1215,468.7745,21.4612,0.6020,0.9234,8.1862,0.0230
br,Bayesian Ridge,18.2544,673.9029,25.8316,0.4250,1.0270,9.2832,0.0090
ridge,Ridge Regression,18.4728,683.7948,26.0165,0.4171,1.0406,9.6375,0.0080
omp,Orthogonal Matching Pursuit,21.0806,812.6091,28.3020,0.3160,1.1350,10.9698,0.0070
lasso,Lasso Regression,22.9892,888.2177,29.6112,0.2498,1.2360,14.4040,0.1590


In [19]:
from pycaret.datasets import get_data
data = get_data('insurance')

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [21]:
from pycaret.regression import *
s = setup(data, target = 'charges')

,Description,Value
0,session_id,3750
1,Target,charges
2,Original Data,"(1338, 7)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,4
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(936, 14)"


In [22]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2485.4792,21393519.7702,4545.0700,0.8446,0.3882,0.2647,0.0110
rf,Random Forest Regressor,2610.6134,22739090.1617,4687.6105,0.8343,0.3871,0.2645,0.0870
lightgbm,Light Gradient Boosting Machine,2835.9005,23961066.8710,4829.2867,0.8256,0.4968,0.3198,0.0110
ada,AdaBoost Regressor,3750.8889,25505086.0503,5013.0329,0.8174,0.5358,0.5475,0.0070
et,Extra Trees Regressor,2610.8542,26527500.9759,5072.4048,0.8060,0.4002,0.2519,0.0800
ridge,Ridge Regression,4228.5423,38645870.6000,6178.6240,0.7264,0.5634,0.4111,0.0050
llar,Lasso Least Angle Regression,4213.0760,38624375.1366,6176.9871,0.7264,0.5658,0.4094,0.0050
br,Bayesian Ridge,4225.7514,38650611.9634,6178.9181,0.7263,0.5645,0.4106,0.0050
lasso,Lasso Regression,4215.4111,38648265.0000,6178.5144,0.7261,0.5720,0.4087,0.1520
lr,Linear Regression,4215.0066,38649401.2000,6178.5908,0.7261,0.5741,0.4085,0.2880


In [23]:
print(best)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=3750, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


In [31]:
evaluate_model(best)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [32]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False
